In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 13.7 MB/s 
     |████████████████████████████████| 6.5 MB 50.2 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 31.5 MB/s 
     |████████████████████████████████| 134 kB 52.3 MB/s 
     |████████████████████████████████| 212 kB 57.4 MB/s 
     |████████████████████████████████| 127 kB 53.9 MB/s 
     |████████████████████████████████| 271 kB 66.6 MB/s 
     |████████████████████████████████| 144 kB 69.0 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urlli

In [ ]:
import numpy as np
import pandas as pd
import re
import random
import torch
from tqdm.notebook import tqdm
import transformers
from datasets import load_dataset
import textwrap

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
dataset = load_dataset("huggingartists/morgenshtern")

Using custom data configuration default


Generating train split: 0 examples [00:00, ? examples/s]

Dataset lyrics_dataset downloaded and prepared to /root/.cache/huggingface/datasets/huggingartists___lyrics_dataset/default/1.0.0/b0f9f2024f227d62378d568126feebf0670d00d78a109bc27b49f9d2fe443bc8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [ ]:
dataset = ' '.join(dataset['train']['text'])

In [ ]:
tokens = tokenizer.encode(dataset, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))

78086


In [ ]:
tokens.shape

(78086,)

In [ ]:
from transformers import GPT2LMHeadModel, AdamW

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);

In [ ]:
batch_size = 7
max_len = 256
epochs = 13

train = len(tokens)//(batch_size*max_len)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# готовим тензоры для обучения

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    model.train()
    pbar = tqdm(range(train))
    for i in pbar:
        batch_ids = prep_tensors(tokens, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()


epoch 1/13 : training


  0%|          | 0/43 [00:00<?, ?it/s]

epoch 2/13 : training


  0%|          | 0/43 [00:00<?, ?it/s]

epoch 3/13 : training


...

  0%|          | 0/43 [00:00<?, ?it/s]

epoch 12/13 : training


  0%|          | 0/43 [00:00<?, ?it/s]

epoch 13/13 : training


  0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
prompt = 'привет'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=88,
    num_beams=2,
    do_sample=True,
    temperature=1.,
    top_k=5,
    top_p=0.3,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


привет, как дела?  Я не могу поверить, что ты не поверишь Ты не можешь поверить в то, будто я не прав Но я верю в это, и
ты в том числе И я уверен, ты никогда не вернёшься ко мне Ведь я так люблю тебя, но не понимаю Как же я тебя люблю, ведь
я же не виноват Что ты так со мной поступил? Что ты сделал
------------------
привет, как дела? Да, я в ударе, но это не значит, что я не в порядке Я не хочу, чтобы ты знала Что я люблю тебя Но я
хочу сказать тебе Не надо меня жалеть Ведь я так любил твою маму И я любил её, а ты нет Она была слишком молода, и я её
любил А ты слишком стар для таких вещей Ты же знаешь,
------------------
привет, как дела? Я в ударе, я в порядке, всё окей А ты? Ты? А ты в чём?  Да, да, ты не поверишь, но это так Это так, и
я не вру Но это правда, что ты мне не нравишься Что я тебе не нравлюсь? Я не верю И я верю в то, о чём ты говоришь Ведь
я
------------------
привет, я твой друг Я не могу без тебя, но я люблю тебя Ты не можешь без меня, ты не моя судьба  Но я тебя л

In [ ]:
torch.save(model, '../model_weights/morgenshtern.pth')